# Seurat object  
## Load required packages


In [ ]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library('enrichR')

## annotation
library("SCINA")

library("viridis")



## Load seurat object (NORM)


In [ ]:
line <- params$line



In [ ]:
seurat_obj <- readRDS(file = paste0('./results/rds/samples/', line, '_Mca.rds'))

seurat_obj
table(seurat_obj$orig.ident)



##  <span style="color:#296d98;"> Parameters </span>  


In [ ]:
enrichR_db = "KEGG_2021_Human"
nb_maxgenes = 200
pc = 50



# Automated Cell type Annotation  


In [ ]:
as.data.frame(seurat_obj@assays$SCT[,]) -> scina.data




## Oligo/Astro  


In [ ]:
## Some markers of oligo/astro cells
mature_oligo <- c("SOX8", "SOX11", "CLDN11","MBP","SOX10","SOX4","MOG","MYT1", "CNP", "PLP1", "OLIG1", "OLIG2", "NKX2-2", "ERBB3", "UGT8", "SOX17", "GPR17", "TNR")
oligo_progenitors  <- c("PDGFRA", "CSPG4")
astro <- c("SLC1A3", "NFIA","SOX9","GFAP","APOE", "AQP4","ALDH1L1", "FABP7", "TNC")

markers_astro_oligo <- list("astro"=astro, "mature_oligo"=mature_oligo, "oligo_progenitors"=oligo_progenitors)


In [ ]:
SCINA(
  scina.data,
  markers_astro_oligo, 
  max_iter = 100, 
  convergence_n = 10, 
  convergence_rate = 0.999, 
  sensitivity_cutoff = 0.9, 
  rm_overlap=TRUE, 
  allow_unknown=TRUE
) -> scina.results

seurat_obj$scina_labels <- scina.results$cell_labels


In [ ]:
colors <- c(astro='seagreen4', oligo_progenitors = "#297fb8" ,mature_oligo='orangered3', unknown="lightgray")

DimPlot(seurat_obj,reduction = "umap",  group.by = "scina_labels", cols = colors)
DimPlot(seurat_obj,reduction = "tsne",  group.by = "scina_labels", cols = colors)


## Panglao database  
### MCA


In [ ]:
seurat_obj <- RunMCA(seurat_obj)



In [ ]:
DimPlotMC(seurat_obj, reduction = "mca", group.by = 'orig.ident', features = c("APOE"), as.text = T)




### Obtaining brain cell-type gene signatures  


In [ ]:
# download all cell-type gene signatures from panglaoDB
panglao <- read_tsv("https://panglaodb.se/markers/PanglaoDB_markers_27_Mar_2020.tsv.gz")

# restricting the analysis to brain specific gene signatues
panglao_brain <- panglao %>% filter(organ == "Brain")
  ##             panglao %>%  filter(str_detect(species,"Hs"))
  ## To obtain gene signatures for all genes

# restricting to human specific genes
panglao_brain <- panglao_brain %>%  filter(str_detect(species,"Hs"))

# converting dataframes into a list of vectors, which is the format needed as input for CellID
panglao_brain <- panglao_brain %>%  
  group_by(`cell type`) %>%  
  summarise(geneset = list(`official gene symbol`))

brain_signatures <- setNames(panglao_brain$geneset, panglao_brain$`cell type`)



### Per-cell gene signature enrichments  


In [ ]:
# Performing per-cell hypergeometric tests against the gene signature collection
HGT_brain_signatures <- RunCellHGT(seurat_obj, pathways = brain_signatures, dims = 1:pc, n.features = nb_maxgenes)


In [ ]:
HGT_brain_signatures <- readRDS( file= paste0('./results/rds/samples/', line, '_HGT_brain_signatures.rds'))



In [ ]:
# For each cell, assess the signature with the lowest corrected p-value (max -log10 corrected p-value)
brain_signatures_prediction <- rownames(HGT_brain_signatures)[apply(HGT_brain_signatures, 2, which.max)]

# For each cell, evaluate if the lowest p-value is significant
brain_signatures_prediction_signif <- ifelse(apply(HGT_brain_signatures, 2, max)>2, yes = brain_signatures_prediction, "unassigned")

# Save cell type predictions as metadata within the Seurat object
seurat_obj$brain_signatures_prediction <- brain_signatures_prediction_signif



### Plots


In [ ]:
nb <- length(unique(seurat_obj$brain_signatures_prediction))

DimPlot(seurat_obj, reduction = "umap", group.by = "brain_signatures_prediction", cols= c(viridis(nb-1),"gray")) + NoLegend()

## see the legend
DimPlot(seurat_obj, reduction = "umap", group.by = "brain_signatures_prediction", label.size = 4, label = TRUE, cols= c(viridis(nb-1),"gray")) + NoLegend()
DimPlot(seurat_obj, reduction = "umap",  group.by = "brain_signatures_prediction", cols= c(viridis(nb-1),"gray"))


In [ ]:
DimPlot(seurat_obj, reduction = "tsne", group.by = "brain_signatures_prediction", cols= c(viridis(nb-1),"gray")) + NoLegend()

## see the legend
DimPlot(seurat_obj, reduction = "tsne", group.by = "brain_signatures_prediction", label.size = 4, label = TRUE, cols= c(viridis(nb-1),"gray")) + NoLegend()
DimPlot(seurat_obj, reduction = "tsne",  group.by = "brain_signatures_prediction", cols= c(viridis(nb-1),"gray"))



#### Percentage


In [ ]:
tibble(
  cluster = seurat_obj$seurat_clusters,
  cell_type = seurat_obj$brain_signatures_prediction
) %>%
  group_by(cluster,cell_type) %>%
  count() %>%
  group_by(cluster) %>%
  mutate(
    percent=(100*n)/sum(n)
  ) %>%
  ungroup() %>%
  mutate(
    cluster=paste("Cluster",cluster)
  ) %>%
  ggplot(aes(x="",y=percent, fill=cell_type)) +
  scale_fill_manual(values=c(viridis(nb-1),"gray"))+
  geom_col(width=1) +
  coord_polar("y", start=0) +
  facet_wrap(vars(cluster)) +  
  theme(axis.text.x=element_blank()) +
  xlab(NULL) +
  ylab(NULL) +
  NoLegend()


In [ ]:
tibble(
  cluster = seurat_obj$seurat_clusters,
  cell_type = seurat_obj$brain_signatures_prediction
) %>%
  group_by(cluster,cell_type) %>%
  count() %>%
  group_by(cluster) %>%
  mutate(
    percent=(100*n)/sum(n)
  ) %>%
  ungroup() %>%
  mutate(
    cluster=paste("Cluster",cluster)
  ) %>%
  ggplot(aes(x="",y=percent, fill=cell_type)) +
  scale_fill_manual(values=c(viridis(nb-1),"gray"))+
  geom_col(width=1) +
  coord_polar("y", start=0) +
  facet_wrap(vars(cluster)) +  
  theme(axis.text.x=element_blank()) +
  xlab(NULL) +
  ylab(NULL) 



#### Split by condition


In [ ]:
DimPlot(seurat_obj, reduction = "umap", group.by = "brain_signatures_prediction", split.by = "condition", cols= c(viridis(nb-1),"gray"))+ NoLegend()

DimPlot(seurat_obj, reduction = "umap", group.by = "brain_signatures_prediction", split.by = "condition", label.size = 4, label = TRUE, cols= c(viridis(nb-1),"gray"))+ NoLegend()

DimPlot(seurat_obj, reduction = "umap", group.by="brain_signatures_prediction", split.by = "condition", cols= c(viridis(nb-1),"gray"))+ NoLegend()


In [ ]:
DimPlot(seurat_obj, reduction = "tsne", group.by = "brain_signatures_prediction", split.by = "condition", cols= c(viridis(nb-1),"gray"))+ NoLegend()

DimPlot(seurat_obj, reduction = "tsne", group.by = "brain_signatures_prediction", split.by = "condition", label.size = 4, label = TRUE, cols= c(viridis(nb-1),"gray"))+ NoLegend()

DimPlot(seurat_obj, reduction = "tsne", group.by="brain_signatures_prediction", split.by = "condition", cols= c(viridis(nb-1),"gray"))+ NoLegend()


# Ontology   
**max.genes** = Maximum number of genes to use as input to enrichR.  
**test.use** = wilcox   

##  enrichR    


In [ ]:
lapply(
  levels(seurat_obj$seurat_clusters),
  function(x)DEenrichRPlot(seurat_obj,  ident.1 = x, assay = "SCT", max.genes = nb_maxgenes, return.gene.list = FALSE, enrich.database =  enrichR_db)
)



# Save seurat object  


In [ ]:
saveRDS(seurat_obj, file = paste0("./results/rds/samples/", line, "_Clusters.rds"))

